<a href="https://colab.research.google.com/github/LivenetsTatiana/works/blob/main/MachineLearning/NaiveBayesClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **<center>Наивный байесовский классификатор<center/>**
**Цель**: изучение наивного байесовского классификатора

**Задания:**
1. Изучить файл «spam_or_not_spam.csv»

2. С помощью from sklearn.feature_extraction.text import CountVectorizer нормировать текстовые данные

3. Написать код на чистом Руthon

4. Реализовать НБА с помощью sklearn.naive_bayes

5. Провести доп. классификацию с одним из представленных методов.
(RandomForestClassifier, DecisionTreeClassifier, Support Vector Classification)



# **Теория**

Теорема Байеса:
\begin{aligned}
&p(A \mid B)=\frac{p(B \mid A) p(A)}{p(B)}\\\end{aligned}

P(A | B) – апостериорная вероятность (что A из B истинно)

P(A) – априорная вероятность (независимая вероятность A)

P(B | A) – вероятность данного значения признака при данном классе. (что B из A истинно)

P(B) – априорная вероятность при значении нашего признака. (независимая вероятность B)
Для нескольких переменных:
\begin{aligned}&p\left(y \mid x_1, x_2\right)=\frac{p\left(x_1, x_2 \mid y\right) p(y)}{p\left(x_1, x_2\right)}
\end{aligned}

где x - характеристика объекта, а

y - метка класса


Будем определять является ли сообщение спамом или нет в зависимости от входящих в него слов.

При этом предполагаем, что слова в тексте не зависят друг от друга.

# Reading data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets download -d ozlerhakan/spam-or-not-spam-dataset

  0% 0.00/1.16M [00:00<?, ?B/s]
100% 1.16M/1.16M [00:00<00:00, 144MB/s]


In [ ]:
!unzip spam-or-not-spam-dataset.zip

Archive:  spam-or-not-spam-dataset.zip
  inflating: spam_or_not_spam.csv    


# Data

In [ ]:
# Загружаем библиотеки и данные
import numpy as np
import pandas as pd
from scipy.stats import norm

data =  pd.read_csv("spam_or_not_spam.csv")
data.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2999 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


# Normalization

In [ ]:
data=data.replace(np.nan,'hello how are you') # замена пустых сообщений на 'hello how are you'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# меняем текстовые сообщения на встречаемость слов в них (1 - слово из словаря есть в сообщении, 0 - нет)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['email'])
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


# Naive Bayes without libraries

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) # делим выборку на тренировочную и тестовую
# обучение
def train(X, y):
    messages, n_messages = np.unique(y, return_counts=True)
    message_prob = n_messages / y.shape[0] # вероятность появления сообщения
    n_words = np.zeros((len(messages), X.shape[1])) # количество слов
    # считаем сколько слов каждого вида встречается в спамах и не спамах
    n_words[0] = X[y == 0].sum(axis=0)
    n_words[1] = X[y == 1].sum(axis=0)
    word_prob = (n_words + 1) / (n_messages[:, np.newaxis] + X.shape[1]) # вероятность появления слова
    return messages, message_prob, word_prob
# предсказание
def predict(X, messages, message_prob, word_prob):
    log_probs = np.log(message_prob) + X.dot(np.log(word_prob).T) # вычисляем логарифмы вероятностей
    return messages[np.argmax(log_probs, axis=1)] # возвращаем класс с наибольшим логарифмом вероятности

a, b, c = train(X_train, y_train)
y_pred = predict(X_test, a, b, c)

In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score (y_test,y_pred )

0.8666666666666667

# Naive Bayes using sklearn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X = X.toarray()
X_train, X_test, y_train, y_test = train_test_split(X, data['label'], test_size=0.3, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,

In [ ]:
accuracy_score (y_test,y_pred )

0.9333333333333333

# Support Vector Classification

Пусть некоторые объекты принадлежат к одному из двух классов.

Задача состоит в том, чтобы определить, к какому классу будут принадлежать новые наблюдения.

В случает метода опорных векторов (Supported Vector Machine) точка в пространстве рассматривается как вектор размерности p.

Мы хотим узнать, сможем ли мы разделить данные точки гиперплоскостью размерности (p-1). Такую плоскость назовем плоскостью классификатора.

Конечно, данные могут быть разделены с помощью различных гиперплоскостей. Лучшая гиперплоскость - гиперплоскость, при построении которой разделение и разница между 2 классами максимально.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split , cross_val_score
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import seaborn

In [ ]:
# в Х кладем письма, а в у - флаг спам/не спам
X =  data.iloc[:,0].values
y = data.iloc[:,-1].values

In [ ]:
# предобработка сообщений - делим на слова, убираем часто встерачающиеся слова (stop-words), которые не несут интереса при классификации,
# приводим к одному регистру
nltk.download('stopwords')
corpus =[]
for i in range(len(X)):
    stemmer = PorterStemmer()
    X[i] = X[i].lower()
    X[i] = X[i].split()
    email =  [stemmer.stem(J) for J in X[i] ]
    email = ' '.join(email)
    corpus.append(email)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# создаем SVC
v = CountVectorizer()
X = v.fit_transform(corpus).toarray()

In [ ]:
svc  =  SVC()
score_1 = cross_val_score(svc,X_train,y_train,cv = 10 )
score_1

array([0.93809524, 0.92857143, 0.92380952, 0.92857143, 0.92857143,
       0.92380952, 0.93809524, 0.94285714, 0.93333333, 0.92380952])

In [ ]:
score_1.mean()

0.930952380952381

# **Вывод**

Были построены классификаторы на базе данных писем спам/не спам. При этом сообщения предварительно отнормированы с переводом слов в их  встречаемость (1 - встречается в сообщении, 0 - нет). Первый наивный байесовский классификатор написан без использования сторонних библиотек и дал точность 0.87, второй (тоже наивный байес) - с использованием библиотеки Scikit-Learn и точность получилась 0.93, a SVC дал точность 0.93